# Bachelor project: Building a dataset of links between medical signs using NLP.

In [1]:
import logging
import ipywidgets as widgets
from os.path import join
from IPython.display import display
import pandas
import math
from dateutil import parser as dateparser
from abstracts_to_df import convert_articles_to_dataframe
from tqdm.auto import tqdm
tqdm.pandas()

logger = logging.getLogger('notebook')
logger.setLevel(logging.DEBUG)
logging.basicConfig(level=logging.ERROR)


Enter the directory where data can be found.

In [2]:
datalocation_widget = widgets.Text(value="data")
display(datalocation_widget)

Text(value='data')

Enter subdirectory for this specific session. If nonexistant it will be created.

In [3]:
sessionlocation_widget = widgets.Text(value="hypersomnia")
display(sessionlocation_widget)

def data_dir():
    return datalocation_widget.value

def session_dir():
    return join(datalocation_widget.value, sessionlocation_widget.value)
downloaded_articles_path = join(datalocation_widget.value, sessionlocation_widget.value, "downloaded_articles")


Text(value='hypersomnia')

## The first step is to get a dataset of article abstracts from pubmed.

The query for the subset of abstracts to be downloaded can be specified below. For this project I focused on a small subset of abstracts that are related to Hypersomnia.

In [4]:
from prepare_subset import get_pubmed_articles, open_local_article_set
query = """hasstructuredabstract[All Fields] AND medline[sb] AND "disorders of excessive somnolence"[MeSH Terms] OR ("disorders"[All Fields] AND "excessive"[All Fields] AND "somnolence"[All Fields]) OR "disorders of excessive somnolence"[All Fields] OR "hypersomnia"[All Fields] AND ("2009/03/20"[PDat] : "2019/03/17"[PDat] AND "humans"[MeSH Terms])"""
querywidget = widgets.Textarea(value=query, disabled=False)
display(querywidget)

Textarea(value='hasstructuredabstract[All Fields] AND medline[sb] AND "disorders of excessive somnolence"[MeSH…

In [5]:
get_pubmed_articles(querywidget.value,downloaded_articles_path,1000,1000)

ERROR:root:Error: directory for supplied name already exists. 


Overwrite? [n]n


Downloaded articles are now in the folder chosen above. 

## We now put the abstracts data in a dataframe.

In [6]:
raw_abstracts = convert_articles_to_dataframe(downloaded_articles_path, datalocation_widget.value)

DEBUG:root:Opening set of articles: data/hypersomnia/downloaded_articles.
DEBUG:root:3 files in directory.
DEBUG:root:Opening structured abstracts mappings file.
DEBUG:root:Opened file: data
DEBUG:root:Processed 3116 abstract labels.
INFO:root:Loaded structured labels from text (0.010000s).
DEBUG:root:Opening next file: data/hypersomnia/downloaded_articles/articles_2000
ERROR:root:Section of article "Searching for the daytime impairments of primary insomnia." with pmid 19963414 has no label.
ERROR:root:Section of article "Excessive daytime sleepiness in psychiatric disorders: Prevalence, correlates and clinical significance." with pmid 19963277 has no label.
ERROR:root:Article "Primary insomnia: a disorder of sleep, or primarily one of wakefulness?" with PMID 19962332 has no 'Abstract' key...
DEBUG:root:Article is not in english
ERROR:root:Section of article "Safety and tolerability of rufinamide in children with epilepsy: a pooled analysis of 7 clinical studies." with pmid 19955344 ha

ERROR:root:Section of article "Predictors of residual sleepiness in adequately treated obstructive sleep apnoea patients." with pmid 19357151 has no label.
ERROR:root:Article "From narcolepsy with cataplexy to idiopathic hypersomnia without long sleep time." with PMID 19342293 has no 'Abstract' key...
ERROR:root:Section of article "Hypersomnia across mood disorders: a review and synthesis." with pmid 19269201 has no label.
ERROR:root:Section of article "The adverse consequences of sleep disturbance in pediatric bipolar disorder: implications for intervention." with pmid 19264266 has no label.
ERROR:root:Section of article "CPAP and behavioral therapies in patients with obstructive sleep apnea: effects on daytime sleepiness, mood, and cognitive function." with pmid 19201228 has no label.
ERROR:root:Article "Hypersomnia, REM sleep fragmentation and loss of motivation in a patient with hypothalamic lesions." with PMID 19185536 has no 'Abstract' key...
ERROR:root:Section of article "Narcol

ERROR:root:Section of article "Incidence of narcolepsy after H1N1 influenza and vaccinations: Systematic review and meta-analysis." with pmid 28847694 has no label.
ERROR:root:Article "Diagnosis and Management of Narcolepsy." with PMID 28837992 has no 'Abstract' key...
DEBUG:root:Article is not in english
ERROR:root:Section of article "Refractory hepatic encephalopathy in a patient with hypothyroidism: Another element in ammonia metabolism." with pmid 28811719 has no label.
ERROR:root:Section of article "Kleine-Levin syndrome elicited by encephalopathy with reversible splenial lesion." with pmid 28804979 has no label.
ERROR:root:Article "Patients with central disorders of excessive hypersomnolence and their perioperative risks and anesthetic management." with PMID 28802599 has no 'Abstract' key...
ERROR:root:Section of article "Klein-Levin syndrome versus bipolar disorder not otherwise specified- diagnostic challenges." with pmid 28784386 has no label.
ERROR:root:Section of article "Is

DEBUG:root:Article is not in english
ERROR:root:Section of article "Hypersomnolence, Hypersomnia, and Mood Disorders." with pmid 28243864 has no label.
ERROR:root:Article "Hypersomnia: an overlooked, but not overestimated, sleep disturbance in bipolar disorder." with PMID 28235881 has no 'Abstract' key...
ERROR:root:Section of article "Val158Met polymorphism in the COMT gene is associated with hypersomnia and mental health-related quality of life in a Colombian sample." with pmid 28235603 has no label.
ERROR:root:Section of article "Efficacy and safety of oral baclofen in the management of spasticity: A rationale for intrathecal baclofen." with pmid 28233010 has no label.
ERROR:root:Section of article "Kleine-Levin Syndrome." with pmid 28216256 has no label.
ERROR:root:Section of article "Cannabis use and the course and outcome of major depressive disorder: A population based longitudinal study." with pmid 28214781 has no label.
ERROR:root:Article "Unusually Low Oxyhemoglobin Saturatio

ERROR:root:Section of article "Obstructive Sleep Apnea With Objective Daytime Sleepiness Is Associated With Hypertension." with pmid 27620392 has no label.
ERROR:root:Section of article "Hypersomnolence-hyperkinetic movement disorder in a child with compound heterozygous mutation in 4-aminobutyrate aminotransferase (ABAT) gene." with pmid 27596361 has no label.
ERROR:root:Article "Obstructive sleep apnea, daytime hypersomnolence and cognitive decline: a scary waterfall?" with PMID 27567530 has no 'Abstract' key...
ERROR:root:Section of article "Prevalence and consequences of insomnia in pediatric population." with pmid 27556114 has no label.
ERROR:root:Section of article "The effects of Transcranial Direct Current Stimulation (tDCS) on Idiopathic Hypersomnia: a pilot study." with pmid 27548094 has no label.
ERROR:root:Section of article "Disorders of Excessive Daytime Sleepiness Including Narcolepsy and Idiopathic Hypersomnia." with pmid 27542882 has no label.
ERROR:root:Section of art

...

ERROR:root:Article "Sleep fragmentation, metabolism, and sleepiness." with PMID 23320488 has no 'Abstract' key...
ERROR:root:Article "Multiple sleep latency test: are the practice and interpretation of the test valid?" with PMID 23317932 has no 'Abstract' key...
ERROR:root:Section of article "Sleep medicine pharmacotherapeutics overview: today, tomorrow, and the future (part 2: hypersomnia, parasomnia, and movement disorders)." with pmid 23276849 has no label.
DEBUG:root:Article is not in english
ERROR:root:Article "Vitamin D as an underlying factor in sleep-related issues." with PMID 23243404 has no 'Abstract' key...
ERROR:root:Section of article "Facial expression recognition and emotional regulation in narcolepsy with cataplexy." with pmid 23228163 has no label.
ERROR:root:Section of article "Rapid eye movement sleep behavior disorder and rapid eye movement sleep without atonia in narcolepsy." with pmid 23219054 has no label.
ERROR:root:Article "Modafinil #259." with PMID 23206152 h

ERROR:root:Section of article "The unrelated communication interlude (UCI)--an automatic behavior phenomenon in Parkinson's disease." with pmid 22682973 has no label.
ERROR:root:Section of article "Migraine and sleep disorders." with pmid 22644169 has no label.
ERROR:root:Section of article "Behavioral disturbance in dementia." with pmid 22644311 has no label.
ERROR:root:Article "Sleepiness and fatigue following traumatic brain injury: a clear relationship?" with PMID 22608678 has no 'Abstract' key...
ERROR:root:Section of article "Kleine Levin syndrome in Malaysia: first typical case." with pmid 22605587 has no label.
ERROR:root:Section of article "The acute effects of levetiracetam on nocturnal sleep and daytime sleepiness in patients with partial epilepsy." with pmid 22595355 has no label.
ERROR:root:Section of article "Shift work and the assessment and management of shift work disorder (SWD)." with pmid 22560640 has no label.
ERROR:root:Section of article "Sleep disturbances in Alz

ERROR:root:Article "Panel discussion: can there be a biomarker for sleepiness?" with PMID 22003333 has no 'Abstract' key...
ERROR:root:Article "Finding a research path for the identification of biomarkers of sleepiness." with PMID 22003331 has no 'Abstract' key...
ERROR:root:Section of article "Association of inflammatory markers with cardiovascular risk and sleepiness." with pmid 22003328 has no label.
ERROR:root:Section of article "Potential of proteomics as a bioanalytic technique for quantifying sleepiness." with pmid 22003327 has no label.
ERROR:root:Article "Panel discussion: current status of measuring sleepiness." with PMID 22003325 has no 'Abstract' key...
ERROR:root:Section of article "Behavioral and genetic markers of sleepiness." with pmid 22003324 has no label.
ERROR:root:Section of article "Behavioral biomarkers of sleepiness." with pmid 22003322 has no label.
ERROR:root:Article "Erratum: Sleepiness, quality of life, and sleep maintenance in REM versus NREM sleep-disorder

ERROR:root:Section of article "Kleine-Levin syndrome: the first typical case in Thailand." with pmid 21280539 has no label.
ERROR:root:Section of article "Idiopathic hypersomnia: a report of three adolescent-onset cases in a two-generation family." with pmid 21270467 has no label.
ERROR:root:Article "An unexpected abnormality on the EEG." with PMID 21254723 has no 'Abstract' key...
ERROR:root:Section of article "Relationship of morningness-eveningness questionnaire score to melatonin and sleep timing, body mass index and atypical depressive symptoms in peri- and post-menopausal women." with pmid 21237517 has no label.
ERROR:root:Section of article "Persistent deficits of visual recall in Kleine-Levin syndrome." with pmid 21236684 has no label.
ERROR:root:Section of article "Resolution of hypersomnia following identification and treatment of vitamin d deficiency." with pmid 21206551 has no label.
ERROR:root:Section of article "Memory deficits due to brain injury: unique PET findings and

ERROR:root:Section of article "Current evaluation and management of excessive daytime sleepiness." with pmid 20437926 has no label.
ERROR:root:Section of article "Subjective symptoms in idiopathic hypersomnia: beyond excessive sleepiness." with pmid 20408941 has no label.
ERROR:root:Section of article "Association between symptoms and subtypes of attention-deficit hyperactivity disorder and sleep problems/disorders." with pmid 20408926 has no label.
ERROR:root:Section of article "Familial recurrent hypersomnia: two siblings with Kleine-Levin syndrome and menstrual-related hypersomnia." with pmid 20404354 has no label.
DEBUG:root:Article is not in english
DEBUG:root:Article is not in english
ERROR:root:Found unknown abstract label: 'subjective measures'.
ERROR:root:Found unknown abstract label: 'objective measures'.
ERROR:root:Section of article "Different sleep onset criteria at the multiple sleep latency test (MSLT): an additional marker to differentiate central nervous system (CNS) h

In [7]:
raw_abstracts.to_csv(join(session_dir(), "raw_abstracts.csv"),index=False)

## Let's transform the dates in timestamps

In [8]:
try: 
    raw_abstracts
    logger.debug("Reusing previously defined raw abstracts df")
except NameError: 
    raw_abstracts = pandas.read_csv(join(session_dir(),"raw_abstracts.csv"))
    logger.debug("Loaded raw abstracts from csv")

def parse_dates(date_dict_str):
    if type(date_dict_str) == float: 
        return date_dict_str
    date_dict = eval(str(date_dict_str))
    datestr = date_dict["Month"] + "." + \
        date_dict["Day"] + "." + date_dict["Year"]
    date = dateparser.parse(datestr).timestamp()
    return date

raw_abstracts['date'] = raw_abstracts['date'].map(lambda x: parse_dates(x))

DEBUG:notebook:Reusing previously defined raw abstracts df


In [9]:
raw_abstracts = raw_abstracts.astype(str)
raw_abstracts.pmid = raw_abstracts.pmid.astype(int)
raw_abstracts.date = raw_abstracts.date.astype(float)
raw_abstracts.iloc[0:3]


,background,objective,methods,conclusions,results,title,journal,pmid,date
0,,Previous studies of atypical depression have b...,A random sample of 3016 Chinese adults complet...,Although limited by the lack of detailed infor...,The 12 month prevalence of atypical depression...,Prevalence and correlates of depression with a...,The Australian and New Zealand journal of psyc...,20001414,1.266534e+09
1,,Determine whether treatment of sleep disorders...,Prospective evaluation of unselected traumatic...,Treatment of sleep disorders after TBI may res...,Abnormal sleep studies were found in 22 subjec...,Treatment of sleep disorders after traumatic b...,Journal of clinical sleep medicine : JCSM : of...,19968047,1.262732e+09
2,,This study reviewed the cumulative postmarketi...,We retrospectively reviewed postmarketing spon...,Cumulative postmarketing and clinical experien...,"Approximately 26,000 patients worldwide receiv...",Safety overview of postmarketing and clinical ...,Journal of clinical sleep medicine : JCSM : of...,19968016,1.262732e+09


In [10]:
raw_abstracts.to_csv(join(session_dir(), "timestamped_abstracts.csv"), index=False)



## We now apply a unicode to ascii transformation

In [11]:
from unicode_to_ascii import unicode2ascii, load_unicode_mappings, get_full_text
try:
    timestamped_abstracts = raw_abstracts
    logger.debug("Reusing previously defined DF.")
except NameError:
    timestamped_abstracts = pandas.read_csv(join(session_dir(),"timestamped_abstracts.csv"))
    timestamped_abstracts = timestamped_abstracts.replace(np.nan, '', regex=True)
    logger.debug("Read timestamped abstracts from dataframe.")

DEBUG:notebook:Reusing previously defined DF.


In [12]:
text_columns = [col for col in timestamped_abstracts.columns if col not in ["date", "pmid"]]

In [14]:
mappings = load_unicode_mappings(data_dir())

In [15]:
timestamped_abstracts[text_columns] = timestamped_abstracts[text_columns].progress_applymap(
    lambda x: unicode2ascii(mappings, str(x)))

In [16]:
timestamped_abstracts.iloc[0:3]

,background,objective,methods,conclusions,results,title,journal,pmid,date
0,,Previous studies of atypical depression have b...,A random sample of 3016 Chinese adults complet...,Although limited by the lack of detailed infor...,The 12 month prevalence of atypical depression...,Prevalence and correlates of depression with a...,The Australian and New Zealand journal of psyc...,20001414,1.266534e+09
1,,Determine whether treatment of sleep disorders...,Prospective evaluation of unselected traumatic...,Treatment of sleep disorders after TBI may res...,Abnormal sleep studies were found in 22 subjec...,Treatment of sleep disorders after traumatic b...,Journal of clinical sleep medicine : JCSM : of...,19968047,1.262732e+09
2,,This study reviewed the cumulative postmarketi...,We retrospectively reviewed postmarketing spon...,Cumulative postmarketing and clinical experien...,"Approximately 26,000 patients worldwide receiv...",Safety overview of postmarketing and clinical ...,Journal of clinical sleep medicine : JCSM : of...,19968016,1.262732e+09


In [17]:
timestamped_abstracts.to_csv(join(session_dir(),"ascii_abstracts.csv"), index=False)

## Named Entity Identification

In [18]:
try:
    ascii_abstracts = timestamped_abstracts
    logger.debug("Reusing previous dataframe.")
except NameError:
    ascii_abstracts = pandas.read_csv(
        join(session_dir(), "ascii_abstracts.csv"))
    ascii_abstracts = ascii_abstracts.replace(np.nan, '', regex=True)
    logger.debug("Loaded ascii abstracts from file.")

DEBUG:notebook:Reusing previous dataframe.


In [19]:
ascii_abstracts.iloc[0:3]

,background,objective,methods,conclusions,results,title,journal,pmid,date
0,,Previous studies of atypical depression have b...,A random sample of 3016 Chinese adults complet...,Although limited by the lack of detailed infor...,The 12 month prevalence of atypical depression...,Prevalence and correlates of depression with a...,The Australian and New Zealand journal of psyc...,20001414,1.266534e+09
1,,Determine whether treatment of sleep disorders...,Prospective evaluation of unselected traumatic...,Treatment of sleep disorders after TBI may res...,Abnormal sleep studies were found in 22 subjec...,Treatment of sleep disorders after traumatic b...,Journal of clinical sleep medicine : JCSM : of...,19968047,1.262732e+09
2,,This study reviewed the cumulative postmarketi...,We retrospectively reviewed postmarketing spon...,Cumulative postmarketing and clinical experien...,"Approximately 26,000 patients worldwide receiv...",Safety overview of postmarketing and clinical ...,Journal of clinical sleep medicine : JCSM : of...,19968016,1.262732e+09


### We start by resolving abbreviations that were defined in the abstract.

In [20]:
from abbreviations import schwartz_hearst
def get_abbreviations(text):
    """
    Get list of abbreviations in sentence
    """
    pairs = schwartz_hearst.extract_abbreviation_definition_pairs(
        doc_text=text)
    return pairs

def get_abstract_abbreviations(art):
    """
    Get all abbreviations defined in abstract. Return a dictionnary of mappings from abbreviation to expanded form
    """
    return get_abbreviations(" ".join(art.iloc[0:6]))
    
def replace_abbreviations(txt, abbs):
    res = txt
    for abb in abbs:
        res = res.replace(abb,abbs[abb])
    return res
    
def replace_abstract_abbreviations(art):
    abbs = get_abstract_abbreviations(art)
    return art.iloc[0:6].map(lambda a: replace_abbreviations(a,abbs))

In [21]:
unabbreviated_abstracts = ascii_abstracts.progress_apply(axis=1,func=replace_abstract_abbreviations)

DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 Omitting candidate 200 mg. Reason: There are less keys in the tokens in front of candidate than there are in the candidate
DEBUG:abbreviations.schwartz_hearst:0 Omitting candidate 0.375 mg. Reason: There are less keys in the tokens in front of candidate than there are in the candidate
DEBUG:abbreviations.schwartz_hearst:11 abbreviations detected and kept (2 omitted)
DEBUG:abbreviations.schwartz_hearst:0 Omitting candidate Xyrem. Reason: There are less keys in the tokens in front of candidate than there are in the candidate
DEBUG:abbreviations.schwartz_hearst:0 Omitting candidate Xyrem. Reason: There are less keys in the tokens in front of candidate than there are in the candidate
DEBUG:abbreviations.schwartz_hearst:1 abbreviations detected and kept (2 omitted)
DEBUG:abbreviations.schwart

DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 Error processing sentence The objective of this study was to evaluate the prevalence of self-reported sleep-disordered breathing (SDB symptoms and its associated risk factors in Pakistani employed adults. Full-time employees (n = 3470) of a medical university were evaluated. Self-administered questionnaire elicited information about demographic data, symptoms of SDB, smoking and alcohol use, presence of nasal congestion, family history of snoring, and included the Epworth Sleepiness Scale. Prevalence of self-reported snoring and of combined SDB symptoms (snoring plus at least one other SDB symptom) was evaluated. Regression an

DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:2 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:4 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:3 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviati

DEBUG:abbreviations.schwartz_hearst:1 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:2 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:2 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 Omitting candidate ESS. Reason: string index out of range
DEBUG:abbreviations.schwartz_hearst:0 Omitting candidate MSLT. Reason: string index out of range
DEBUG:abbreviations.schwartz_hearst:0 Omitting candidate REM. Reason: string index out of range
DEBUG:abbreviations.schwartz_hearst:0 abbreviations det

DEBUG:abbreviations.schwartz_hearst:1 Omitting candidate p=.527. Reason: string index out of range
DEBUG:abbreviations.schwartz_hearst:3 abbreviations detected and kept (1 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:1 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 Omitting definition Controlled Trials for candidate CENTRAL. Reason: string index out of range
DEBUG:abbreviations.schwartz_hearst:4 abbreviations detected and kept (1 omitted)
DEBUG:abbreviations.schwartz_hearst:2 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:1 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 Omitting definition 2010. We explored the utility of ecological data to as

...

DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:2 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 Omitting definition ESS was obtained as usual in the laboratory routine, immediately before the sleep study for candidate ESS1. Reason: string index out of range
DEBUG:abbreviations.schwartz_hearst:0 Omitting definition ESS was obtained as usual in the laboratory routine, immediately before the sleep study (ESS1) and was repeated in the morning, after the polysomnography for candidate ESS2. Reason: string index out of range
DEBUG:abbreviations.schwartz_hearst:0 Omitting candidate ESS1. Reason: string index out of range
DEBUG:abbreviations.schwart

DEBUG:abbreviations.schwartz_hearst:0 Omitting candidate NS. Reason: string index out of range
DEBUG:abbreviations.schwartz_hearst:0 Omitting candidate France. Reason: string index out of range
DEBUG:abbreviations.schwartz_hearst:1 abbreviations detected and kept (5 omitted)
DEBUG:abbreviations.schwartz_hearst:0 Omitting candidate referent. Reason: string index out of range
DEBUG:abbreviations.schwartz_hearst:1 abbreviations detected and kept (1 omitted)
DEBUG:abbreviations.schwartz_hearst:0 Omitting definition obstructive sleep apnea for candidate OSAS. Reason: string index out of range
DEBUG:abbreviations.schwartz_hearst:0 Omitting candidate REM. Reason: string index out of range
DEBUG:abbreviations.schwartz_hearst:2 abbreviations detected and kept (2 omitted)
DEBUG:abbreviations.schwartz_hearst:0 Omitting candidate RMT. Reason: string index out of range
DEBUG:abbreviations.schwartz_hearst:0 Omitting candidate MEP. Reason: string index out of range
DEBUG:abbreviations.schwartz_hearst

DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:2 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 Omitting definition against Tribbles homolog 2 for candidate anti-TRIB2. Reason: string index out of range
DEBUG:abbreviations.schwartz_hearst:0 Omitting candidate TRIB2. Reason: string index out of range
DEBUG:abbreviations.schwartz_hearst:2 abbreviations detected and kept (2 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:1 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 Omitting candidate QID. Reason: There are less keys in the tokens in front of candidate than there are in the candidate
DEBUG:abbreviations.schwa

DEBUG:abbreviations.schwartz_hearst:2 abbreviations detected and kept (1 omitted)
DEBUG:abbreviations.schwartz_hearst:2 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:2 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:1 Omitting candidate 704 min. Reason: There are less keys in the tokens in front of candidate than there are in the candidate
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (1 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:2 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 abbreviations detected and kept (0 omitted)
DEBUG:abbreviation

DEBUG:abbreviations.schwartz_hearst:0 Omitting definition -reported sleep disruption. AHI(REM) was not associated with the ESS scores or the physical and mental components scales scores of the SF-36 after adjusting for demographics, body mass index, and AHI(NREM) x AHI for candidate REM. Reason: did not meet min(|A|+5, |A|*2) constraint
DEBUG:abbreviations.schwartz_hearst:0 Omitting definition not associated with frequent difficulty maintaining sleep or early awakening from sleep. AHI for candidate NREM. Reason: did not meet min(|A|+5, |A|*2) constraint
DEBUG:abbreviations.schwartz_hearst:0 Omitting candidate REM. Reason: string index out of range
DEBUG:abbreviations.schwartz_hearst:4 abbreviations detected and kept (7 omitted)
DEBUG:abbreviations.schwartz_hearst:7 abbreviations detected and kept (0 omitted)
DEBUG:abbreviations.schwartz_hearst:0 Omitting definition saturation for candidate SaO(2). Reason: string index out of range
DEBUG:abbreviations.schwartz_hearst:0 Omitting definiti

In [22]:
unabbreviated_abstracts.to_csv(join(session_dir(), "unabbreviated_abstracts.csv"),index=False)

### We need to do NER.

Metamap is extremely slow: ~5s per abstract, which amounts to 2.5months of uninterrupted processing for the whole set of pubmed abstracts (1.5M abstracts)... 

Using IMADA's supercomputer, I could probably get this to about 3 days assuming uninterrupted running on all 32 cpus... meh?

SOPHIA: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4420011/#!po=87.5000 - much faster but more geared towards clinical notes. 

Viable options:

* QuickUMLS - https://github.com/Georgetown-IR-Lab/QuickUMLS